In [1]:
import pandas as pd

byYear = pd.read_csv('./queryResults/byYear.csv', 
                       dtype={'jumbo':'bool'})

In [2]:
byYear

,year,jumbo,amount_loaned,loans,lenders
0,2018,False,878997120000,3872328,5478
1,2018,True,283352865000,336593,4026
2,2019,False,1257705575000,4952743,5354
3,2019,True,389338135000,423993,3947
4,2020,False,2626364080000,9572302,4357
5,2020,True,491308090000,527980,3549
6,2021,False,2740499655000,9803255,4217
7,2021,True,643361895000,584605,3260
8,2022,False,1286809110000,4292794,4309
9,2022,True,377866645000,295477,3122


In [3]:
byYear.groupby(['year', 'jumbo']).sum()

amount_loaned    loans  lenders
year jumbo                                 
2018 False   878997120000  3872328     5478
     True    283352865000   336593     4026
2019 False  1257705575000  4952743     5354
     True    389338135000   423993     3947
2020 False  2626364080000  9572302     4357
     True    491308090000   527980     3549
2021 False  2740499655000  9803255     4217
     True    643361895000   584605     3260
2022 False  1286809110000  4292794     4309
     True    377866645000   295477     3122

In [4]:
df = pd.DataFrame()
# jumbo
df['year'] = byYear.year.unique()
df['jumbo_loans'] = byYear.loc[byYear.jumbo==True]['loans'].to_list()
df['jumbo_amount_loaned'] = byYear.loc[byYear.jumbo==True]['amount_loaned'].to_list()
df['jumbo_lenders'] = byYear.loc[byYear.jumbo==True]['lenders'].to_list()

# conforming
df['conf_loans'] = byYear.loc[byYear.jumbo==False]['loans'].to_list()
df['conf_amount_loaned'] = byYear.loc[byYear.jumbo==False]['amount_loaned'].to_list()

In [5]:
df['all_loans'] = df.jumbo_loans + df.conf_loans
df['all_amount_loaned'] = df.jumbo_amount_loaned + df.conf_amount_loaned
df['perc_loans_jumbo'] = df.jumbo_loans / df.all_loans
df['perc_amount_jumbo'] = df.jumbo_amount_loaned / df.all_amount_loaned
df 

,year,jumbo_loans,jumbo_amount_loaned,jumbo_lenders,conf_loans,conf_amount_loaned,all_loans,all_amount_loaned,perc_loans_jumbo,perc_amount_jumbo
0,2018,336593,283352865000,4026,3872328,878997120000,4208921,1162349985000,0.079971,0.243776
1,2019,423993,389338135000,3947,4952743,1257705575000,5376736,1647043710000,0.078857,0.236386
2,2020,527980,491308090000,3549,9572302,2626364080000,10100282,3117672170000,0.052274,0.157588
3,2021,584605,643361895000,3260,9803255,2740499655000,10387860,3383861550000,0.056278,0.190127
4,2022,295477,377866645000,3122,4292794,1286809110000,4588271,1664675755000,0.064398,0.226991


In [6]:
byLender = pd.read_csv('./queryResults/byLEI_2022.csv', 
                       dtype={'jumbo':'bool'})                  

In [7]:
byLender 

,lei,jumbo,respondent_name,amount_loaned,loans
0,01ERPZV3DOLNXY2MLB90,False,Libertyville Bank & Trust NA,33370000,82
1,01ERPZV3DOLNXY2MLB90,True,Libertyville Bank & Trust NA,14095000,13
2,01J4SO3XTWZF4PP38209,False,TRUSTMARK NATIONAL BANK,1375250000,5652
3,01J4SO3XTWZF4PP38209,True,TRUSTMARK NATIONAL BANK,158675000,161
4,01KWVG908KE7RKPTNP46,False,HomeStreet Bank,522730000,1214
...,...,...,...,...,...
7426,Z867SNMO7WGY8TTGQG78,True,"SUMMIT COMMUNITY BANK, INC",10180000,10
7427,ZF85QS7OXKPBG52R7N18,False,Associated Bank NA,1316510000,4856
7428,ZF85QS7OXKPBG52R7N18,True,Associated Bank NA,1322125000,1341
7429,ZXMJHJK466PBZTM5F379,False,Raymond James Bank,265165000,605


In [8]:
df = byLender.loc[byLender.jumbo==False].merge(byLender.loc[byLender.jumbo==True], how='outer', on='lei').drop(columns=['jumbo_x', 'jumbo_y', 'respondent_name_y']).rename(
    columns={'respondent_name_x':'respondent_name',
             'amount_loaned_x':'conf_amount_loaned',
             'amount_loaned_y':'jumbo_amount_loaned',
             'loans_x':'conf_loans',
             'loans_y':'jumbo_loans'}
)
df

,lei,respondent_name,conf_amount_loaned,conf_loans,jumbo_amount_loaned,jumbo_loans
0,01ERPZV3DOLNXY2MLB90,Libertyville Bank & Trust NA,3.337000e+07,82.0,1.409500e+07,13.0
1,01J4SO3XTWZF4PP38209,TRUSTMARK NATIONAL BANK,1.375250e+09,5652.0,1.586750e+08,161.0
2,01KWVG908KE7RKPTNP46,HomeStreet Bank,5.227300e+08,1214.0,3.739250e+08,319.0
3,03D0JEWFDFUS0SEEKG89,TD Bank,3.003970e+09,7558.0,3.543655e+09,2663.0
4,0K2D5AK28E3O5CC06E35,Silicon Valley Bank,2.615100e+08,548.0,1.858750e+09,844.0
...,...,...,...,...,...,...
4305,YWC0TIKBQM2JV8L4IV08,FIRST REPUBLIC BANK,5.101710e+09,8514.0,2.051162e+10,10021.0
4306,Z867SNMO7WGY8TTGQG78,"SUMMIT COMMUNITY BANK, INC",5.538000e+07,310.0,1.018000e+07,10.0
4307,ZF85QS7OXKPBG52R7N18,Associated Bank NA,1.316510e+09,4856.0,1.322125e+09,1341.0
4308,ZXMJHJK466PBZTM5F379,Raymond James Bank,2.651650e+08,605.0,1.058915e+09,645.0


In [9]:
df['all_loans'] = df.conf_loans + df.jumbo_loans 
df['all_amount_loaned'] = df.conf_amount_loaned + df.jumbo_amount_loaned
df['perc_loans_jumbo'] = df.jumbo_loans / df.all_loans
df['perc_amount_jumbo'] = df.jumbo_amount_loaned / df.all_amount_loaned

In [10]:
df.sort_values(by='jumbo_loans', ascending=False).head(10)

,lei,respondent_name,conf_amount_loaned,conf_loans,jumbo_amount_loaned,jumbo_loans,all_loans,all_amount_loaned,perc_loans_jumbo,perc_amount_jumbo
4241,KB1H1DSPRFMYMCUFXT09,WELLS FARGO BANK NA,3.247388e+10,110453.0,2.866225e+10,19896.0,130349.0,6.113612e+10,0.152636,0.468827
4205,B4TYDEB6GKMZO031MB27,Bank of America NA,1.488402e+10,43043.0,2.044667e+10,13618.0,56661.0,3.533068e+10,0.240342,0.578723
4160,6BYL5QZYBDK8S7L73M02,"US BANK, N.A.",1.395718e+10,46081.0,1.597998e+10,12589.0,58670.0,2.993715e+10,0.214573,0.533784
4167,7H6GLXDRUGQFU57RNE97,"JPMorgan Chase Bank, NA",2.510240e+10,82959.0,1.794866e+10,12223.0,95182.0,4.305106e+10,0.128417,0.416916
4305,YWC0TIKBQM2JV8L4IV08,FIRST REPUBLIC BANK,5.101710e+09,8514.0,2.051162e+10,10021.0,18535.0,2.561332e+10,0.540653,0.800818
4220,E57ODZWZ7FF32TWEFA76,"Citibank, NA",3.564830e+09,7946.0,1.375994e+10,7299.0,15245.0,1.732476e+10,0.478780,0.794235
2436,549300HW662MN1WU8550,United Wholesale Mortgage,8.002666e+10,225401.0,7.043575e+09,6693.0,232094.0,8.707024e+10,0.028837,0.080895
4204,AD6GFRVSDT01YPT1CS68,PNC BANK N.A.,5.925010e+09,20194.0,7.908525e+09,5597.0,25791.0,1.383354e+10,0.217014,0.571692
2328,549300GS0W0TEUQS8571,Morgan Stanley Private Bank NA,2.649825e+09,4919.0,7.516675e+09,4603.0,9522.0,1.016650e+10,0.483407,0.739357
3595,549300U3721PJGQZYY68,"GUARANTEED RATE, INC",1.878950e+10,53034.0,4.777630e+09,4414.0,57448.0,2.356713e+10,0.076835,0.202724


In [11]:
df.sort_values(by='jumbo_amount_loaned', ascending=False).head(10)

,lei,respondent_name,conf_amount_loaned,conf_loans,jumbo_amount_loaned,jumbo_loans,all_loans,all_amount_loaned,perc_loans_jumbo,perc_amount_jumbo
4241,KB1H1DSPRFMYMCUFXT09,WELLS FARGO BANK NA,3.247388e+10,110453.0,2.866225e+10,19896.0,130349.0,6.113612e+10,0.152636,0.468827
4305,YWC0TIKBQM2JV8L4IV08,FIRST REPUBLIC BANK,5.101710e+09,8514.0,2.051162e+10,10021.0,18535.0,2.561332e+10,0.540653,0.800818
4205,B4TYDEB6GKMZO031MB27,Bank of America NA,1.488402e+10,43043.0,2.044667e+10,13618.0,56661.0,3.533068e+10,0.240342,0.578723
4167,7H6GLXDRUGQFU57RNE97,"JPMorgan Chase Bank, NA",2.510240e+10,82959.0,1.794866e+10,12223.0,95182.0,4.305106e+10,0.128417,0.416916
4160,6BYL5QZYBDK8S7L73M02,"US BANK, N.A.",1.395718e+10,46081.0,1.597998e+10,12589.0,58670.0,2.993715e+10,0.214573,0.533784
4220,E57ODZWZ7FF32TWEFA76,"Citibank, NA",3.564830e+09,7946.0,1.375994e+10,7299.0,15245.0,1.732476e+10,0.478780,0.794235
4204,AD6GFRVSDT01YPT1CS68,PNC BANK N.A.,5.925010e+09,20194.0,7.908525e+09,5597.0,25791.0,1.383354e+10,0.217014,0.571692
2328,549300GS0W0TEUQS8571,Morgan Stanley Private Bank NA,2.649825e+09,4919.0,7.516675e+09,4603.0,9522.0,1.016650e+10,0.483407,0.739357
2436,549300HW662MN1WU8550,United Wholesale Mortgage,8.002666e+10,225401.0,7.043575e+09,6693.0,232094.0,8.707024e+10,0.028837,0.080895
4259,OX3PU53ZLPQKJ4700D47,"MUFG Union Bank, NA",2.110975e+09,3721.0,5.855415e+09,3939.0,7660.0,7.966390e+09,0.514230,0.735015


In [12]:
df.loc[df.jumbo_loans >= 100].sort_values(by='perc_loans_jumbo', ascending=False).head(10)

,lei,respondent_name,conf_amount_loaned,conf_loans,jumbo_amount_loaned,jumbo_loans,all_loans,all_amount_loaned,perc_loans_jumbo,perc_amount_jumbo
4242,KD3XUN7C6T14HNAYLU02,Goldman Sachs Bank USA,49560000.0,72.0,1.248320e+09,576.0,648.0,1.297880e+09,0.888889,0.961815
2175,549300F6Y1YO2YXN6135,Redwood Residential Acquisition Corp,28050000.0,40.0,2.520000e+08,234.0,274.0,2.800500e+08,0.854015,0.899839
2500,549300INFJ8TYNZ1G568,"CHARLES SCHWAB BANK, SSB",566855000.0,1229.0,4.138760e+09,3168.0,4397.0,4.705615e+09,0.720491,0.879536
4162,6PTKHDJ8HDUF78PFWH30,The Northern Trust Co,49390000.0,110.0,7.767450e+08,279.0,389.0,8.261350e+08,0.717224,0.940216
793,4EP6JBYBTPTQ47LZOB67,"BNY MELLON, N.A.",128570000.0,224.0,1.025570e+09,520.0,744.0,1.154140e+09,0.698925,0.888601
3499,549300T7QCXC3PXO0W07,Amplify Credit Union,91935000.0,267.0,5.508850e+08,557.0,824.0,6.428200e+08,0.675971,0.856982
4,0K2D5AK28E3O5CC06E35,Silicon Valley Bank,261510000.0,548.0,1.858750e+09,844.0,1392.0,2.120260e+09,0.606322,0.876661
2339,549300GWD9H4FQ2VR805,UBS BANK USA,692085000.0,1309.0,3.328875e+09,1807.0,3116.0,4.020960e+09,0.579910,0.827881
871,5493000ZB0IHPJ66BJ32,LUTHER BURBANK SAVINGS,150700000.0,246.0,5.634000e+08,310.0,556.0,7.141000e+08,0.557554,0.788965
4157,593C3GZG957YOJPS2Z63,CITY NATIONAL BANK,770920000.0,1442.0,3.959800e+09,1816.0,3258.0,4.730720e+09,0.557397,0.837040


In [13]:
df.loc[df.jumbo_loans >= 100].sort_values(by='perc_amount_jumbo', ascending=False).head(10)

,lei,respondent_name,conf_amount_loaned,conf_loans,jumbo_amount_loaned,jumbo_loans,all_loans,all_amount_loaned,perc_loans_jumbo,perc_amount_jumbo
4242,KD3XUN7C6T14HNAYLU02,Goldman Sachs Bank USA,49560000.0,72.0,1.248320e+09,576.0,648.0,1.297880e+09,0.888889,0.961815
4162,6PTKHDJ8HDUF78PFWH30,The Northern Trust Co,49390000.0,110.0,7.767450e+08,279.0,389.0,8.261350e+08,0.717224,0.940216
2175,549300F6Y1YO2YXN6135,Redwood Residential Acquisition Corp,28050000.0,40.0,2.520000e+08,234.0,274.0,2.800500e+08,0.854015,0.899839
793,4EP6JBYBTPTQ47LZOB67,"BNY MELLON, N.A.",128570000.0,224.0,1.025570e+09,520.0,744.0,1.154140e+09,0.698925,0.888601
2500,549300INFJ8TYNZ1G568,"CHARLES SCHWAB BANK, SSB",566855000.0,1229.0,4.138760e+09,3168.0,4397.0,4.705615e+09,0.720491,0.879536
4,0K2D5AK28E3O5CC06E35,Silicon Valley Bank,261510000.0,548.0,1.858750e+09,844.0,1392.0,2.120260e+09,0.606322,0.876661
4229,GDN5JXQ4P5NZE28ZRC90,CITY NATIONAL BANK OF FLORIDA,178630000.0,492.0,1.185465e+09,483.0,975.0,1.364095e+09,0.495385,0.869049
3499,549300T7QCXC3PXO0W07,Amplify Credit Union,91935000.0,267.0,5.508850e+08,557.0,824.0,6.428200e+08,0.675971,0.856982
4157,593C3GZG957YOJPS2Z63,CITY NATIONAL BANK,770920000.0,1442.0,3.959800e+09,1816.0,3258.0,4.730720e+09,0.557397,0.837040
2339,549300GWD9H4FQ2VR805,UBS BANK USA,692085000.0,1309.0,3.328875e+09,1807.0,3116.0,4.020960e+09,0.579910,0.827881


In [14]:
byCensus = pd.read_csv('./queryResults/byCensusTract_2022.csv', 
                       dtype={'jumbo':'bool'})
byCensus['census_tract'] = [f'{x:0>11}' for x in byCensus.census_tract]

In [15]:
byCensus.dropna(subset='state', inplace=True)
byCensus

,census_tract,state,jumbo,amount_loaned,loans
2,01001020100,AL,False,3680000,18
3,01001020200,AL,False,2280000,14
4,01001020300,AL,False,7725000,45
5,01001020400,AL,False,10275000,63
6,01001020501,AL,False,18690000,86
...,...,...,...,...,...
112648,78010970600,VI,False,550000,2
112649,78010970700,VI,False,365000,1
112650,78010971000,VI,False,620000,2
112651,78010971500,VI,False,575000,3


In [16]:
df = byCensus.loc[byCensus.jumbo==False].groupby('state').sum().merge(byCensus.loc[byCensus.jumbo==True].groupby('state').sum(), how='outer', on='state')\
    .drop(columns=['jumbo_x', 'jumbo_y', 'census_tract_x', 'census_tract_y']).rename(
    columns={'amount_loaned_x':'conf_amount_loaned',
             'amount_loaned_y':'jumbo_amount_loaned',
             'loans_x':'conf_loans',
             'loans_y':'jumbo_loans'}
)
df


,conf_amount_loaned,conf_loans,jumbo_amount_loaned,jumbo_loans
state,,,,
AK,2411405000,7849,4.144500e+07,35.0
AL,14491015000,63769,1.640395e+09,1677.0
AR,7534770000,34954,6.310200e+08,666.0
AZ,40646185000,126399,8.650755e+09,7939.0
CA,174182135000,381679,1.174857e+11,77844.0
CO,37859945000,98779,1.430900e+10,12397.0
CT,12677555000,44579,6.176830e+09,4644.0
DC,4071605000,8335,1.949585e+09,1357.0
DE,4855015000,16455,7.344650e+08,715.0


In [17]:
df['all_loans'] = df.conf_loans + df.jumbo_loans 
df['all_amount_loaned'] = df.conf_amount_loaned + df.jumbo_amount_loaned
df['perc_loans_jumbo'] = df.jumbo_loans / df.all_loans
df['perc_amount_jumbo'] = df.jumbo_amount_loaned / df.all_amount_loaned

In [18]:
for item in df.loc['CA'].items():
    print(item)

('conf_amount_loaned', 174182135000.0)
('conf_loans', 381679.0)
('jumbo_amount_loaned', 117485740000.0)
('jumbo_loans', 77844.0)
('all_loans', 459523.0)
('all_amount_loaned', 291667875000.0)
('perc_loans_jumbo', 0.16940174920515405)
('perc_amount_jumbo', 0.40280658265844327)
